In [14]:
import numpy as np
from numpy.testing import assert_almost_equal
import scipy.integrate as integrate
import matplotlib.pyplot as plt 
rng = np.random.default_rng(seed = 42)

In [9]:
### Constants
sigma1 = 0.11
sigma2 = 0.13
T = 1.7
mu = 0.13
r = 0.01
S0 = 149
time_break = 0.3
K = 188
KH = S0

In [32]:
### Helper Functions

def time_varying_vol(t):
    if t > time_break:
        return sigma1 + sigma2 * (t - time_break)/(T - time_break)
    return sigma1

assert_almost_equal(np.array([time_varying_vol(0.2), time_varying_vol(0.4), time_varying_vol(T)]) ,
                    np.array([sigma1, 0.11928571428, sigma1 + sigma2]))

def integrated_vol(t):
    squared_vol = lambda x: time_varying_vol(x) ** 2
    result = integrate.quad(squared_vol, 0, t)
    return result[0]

assert_almost_equal(np.array([integrated_vol(0.3), integrated_vol(1), integrated_vol(1.7)]),
                    np.array([0.3 * 0.11 ** 2, 0.018090833333, 0.0484766666]))

